# One-Way ANOVA (일원분산분석)
## 1. 작업 준비
#### 패키지 가져오기

In [2]:
import os
import sys
from pandas import read_excel, melt
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner, levene, f_oneway, chi2_contingency
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import normality_test, equal_variance_test, independence_test, all_test

## (예제 1)
- 어떤 지역에서 동일한 제품을 판매하고 있는 두 가게에 대한 일별 매출액
### 1. 데이터 가져오기

In [3]:
df = read_excel("https://data.hossam.kr/E02/store.xlsx")
df

,store1,store2
0,46,78
1,47,57
2,58,31
3,47,28
4,27,67
5,58,77
6,56,36
7,26,57
8,47,36
9,25,57


In [4]:
df2 = df.melt(var_name='store', value_name='sales')
df2['store'] = df2['store'].map({'store1': 0, 'store2': 1})
df2

,store,sales
0,0,46
1,0,47
2,0,58
3,0,47
4,0,27
5,0,58
6,0,56
7,0,26
8,0,47
9,0,25


### 2. 분산분석의 조건 충족 여부 검사
#### 1) 데이터의 정규성 검정
##### shapiro wilk 검정
- `shapiro(DataFrame['컬럼명'])`
- `샘플의 수가 적을 때` `정규성을 확인`하는 검정
- (샘플의 수가 대략 50개 미만인 경우, 중심극한 정리는 30개 미만을 권장하기도 함)
|가설|내용|
|--:|--|
|귀무가설|집단간 데이터 분포에는 차이가 없다(정규성을 따름)|
|대립가설|집단간 데이터 분포에는 차이가 있다(정규성을 따르지 않음)|

In [5]:
print(shapiro(df2['store']))
print(shapiro(df2['sales']))

ShapiroResult(statistic=0.6411193013191223, pvalue=8.099757906165905e-06)
ShapiroResult(statistic=0.9331830739974976, pvalue=0.17777423560619354)


##### Normal Test
- `normaltest(DataFrame['컬럼명'])`

In [6]:
print(normaltest(df2['store']))
print(normaltest(df2['sales']))

NormaltestResult(statistic=52.67832231762539, pvalue=3.6395496705596064e-12)
NormaltestResult(statistic=0.8935420573562478, pvalue=0.6396903622551172)


##### 콜모고로프-스미르노프 검정(Kolmogorov-Smirnov test)
- `ks_2samp()`
- 정규분포에 국한되지 않고 `두 표본이 같은 분포를 따르는지 확인`할 수 있는 방법
- 한 번에 `2개씩 검사` 가능

In [7]:
ks_2samp(df2['store'], df2['sales'])

KstestResult(statistic=1.0, pvalue=1.4508889103849681e-11, statistic_location=1, statistic_sign=1)

##### 함수로 한 번에 처리하기
- `normality_test()`는 helper.py 모듈에 정의한 함수

In [8]:
normality_test(df2['store'], df2['sales'])

statistic       p-value  result
condition test       field                                          
정규성       shapiro    store            0.641119  8.099758e-06   False
                     sales            0.933183  1.777742e-01    True
          normaltest store           52.678322  3.639550e-12   False
                     sales            0.893542  6.396904e-01    True
          ks_2samp   store vs sales   1.000000  1.450889e-11   False
                     sales vs store   1.000000  1.450889e-11   False

#### 2) 데이터의 등분산성 검정
##### Bartlett 검정
- `bartlett()`
- 집단간 `분산이 같은지 다른지 여부를 알아볼 때` 사용
- 독립 2표본 t-검정 또는 일원분산분석(one-way ANOVA) 실시 전에 `등분산성을 확인하는 용도`
- Barlett 검정은 두 집단 뿐만 아니라 `세 집단 이상`에서도 사용 가능
- `모든 변수가 정규분포를 따른다`는 가정 하에서만 사용 가능

|가설|내용|
|--:|--|
|귀무가설|집단간 분산이 차이가 없다(같다)|
|대립가설|집단간 분산이 차이가 있다(다르다)|

In [9]:
bartlett(df2['store'], df2['sales'])

BartlettResult(statistic=102.42750228837312, pvalue=4.474464390522148e-24)

##### fligner 검정
- `fligner()`
- Fligner-Killeen test는 비모수 등분산 검정으로 `각 독립 표본들이 정규분포를 따르지 않아도 사용 가능`한 검정 방법

|가설|내용|
|--:|--|
|귀무가설|집단간 분산이 차이가 없다(같다)|
|대립가설|집단간 분산이 차이가 있다(다르다)|

In [10]:
fligner(df2['store'], df2['sales'])

FlignerResult(statistic=16.545710139110323, pvalue=4.749130117443737e-05)

##### 레빈 검정 (Levene's test)
- `levene()`
- 통계학에서 `등분산성(homoscedasticity)을 검증하기 위해 사용`되는 방법
- 분석하려는 `데이터의 그룹이 2개 이상`인 경우 사용
- 다른 등분산성 검정 방법과 달리 레빈 검정은 정규성 가정을 거의 하지 않기에 `비모수적인 방법으로도 적용 가능`

|가설|내용|
|--:|--|
|귀무가설|집단간 분산이 차이가 없다(같다)|
|대립가설|집단간 분산이 차이가 있다(다르다)|

In [11]:
levene(df2['store'], df2['sales'])

LeveneResult(statistic=37.756387569463854, pvalue=3.605670265394161e-07)

##### 함수로 한 번에 처리하기
- `equal_variance_test()`는 helper.py 모듈에 정의한 함수

In [12]:
y = equal_variance_test(df2['store'], df2['sales'])
y

statistic       p-value  result
condition test     field                                           
등분산성      Bartlett store vs sales  102.427502  4.474464e-24   False
          Fligner  store vs sales   16.545710  4.749130e-05   False
          Levene   store vs sales   37.756388  3.605670e-07   False

#### 3) 독립성 검정
- `chi2_contingency()`

|가설|내용|
|--:|--|
|귀무가설|각 변수는 상관이 없다(독립적이다)|
|대립가설|각 변수는 상관이 있다(독립적이지 않다)|

In [13]:
chi2_contingency(df2[['store', 'sales']])

Chi2ContingencyResult(statistic=10.665480367780301, pvalue=0.9345787652103084, dof=19, expected_freq=array([[ 0.47373841, 45.52626159],
       [ 0.48403708, 46.51596292],
       [ 0.59732235, 57.40267765],
       [ 0.48403708, 46.51596292],
       [ 0.27806385, 26.72193615],
       [ 0.59732235, 57.40267765],
       [ 0.57672503, 55.42327497],
       [ 0.26776519, 25.73223481],
       [ 0.48403708, 46.51596292],
       [ 0.25746653, 24.74253347],
       [ 0.81359423, 78.18640577],
       [ 0.59732235, 57.40267765],
       [ 0.32955716, 31.67044284],
       [ 0.29866117, 28.70133883],
       [ 0.70030896, 67.29969104],
       [ 0.80329557, 77.19670443],
       [ 0.38105046, 36.61894954],
       [ 0.59732235, 57.40267765],
       [ 0.38105046, 36.61894954],
       [ 0.59732235, 57.40267765]]))

##### 함수로 처리하기
- `independence_test()`는 helper.py 모듈에 정의한 함수

In [14]:
independence_test(df2['store'], df2['sales'])

,,,statistic,p-value,result
condition,test,field,,,
독립성,Chi2,store vs sales,10.66548,0.934579,True


#### 4) 모든 조건을 하나의 함수로 확인하기
- `all_test()`는 helper.py 모듈에 정의한 함수

In [15]:
all_test(df2['store'], df2['sales'])

statistic       p-value  result
condition test       field                                           
정규성       shapiro    store             0.641119  8.099758e-06   False
                     sales             0.933183  1.777742e-01    True
          normaltest store            52.678322  3.639550e-12   False
                     sales             0.893542  6.396904e-01    True
          ks_2samp   store vs sales    1.000000  1.450889e-11   False
                     sales vs store    1.000000  1.450889e-11   False
등분산성      Bartlett   store vs sales  102.427502  4.474464e-24   False
          Fligner    store vs sales   16.545710  4.749130e-05   False
          Levene     store vs sales   37.756388  3.605670e-07   False
독립성       Chi2       store vs sales   10.665480  9.345788e-01    True

### 3. `scipy.stats` 패키지를 사용한 분산분석
- `f_oneway()`

In [16]:
f_oneway(df2['store'], df2['sales'])

F_onewayResult(statistic=170.06917131008828, pvalue=1.3198634743887305e-15)

##### 해석
- `p-value가 0.05보다 크므로` 귀무가설을 기각할 수 없다
- 즉, `두 가게의 일별 매출 평균은 같다`.

### 4. statsmodels 패키지를 사용한 분산분석
- `ols()`
- `anova_lm()`

In [17]:
lm = ols('sales ~ C(store)', data=df2).fit()
anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
C(store),1.0,378.45,378.450000,1.459162,0.242696
Residual,18.0,4668.50,259.361111,NaN,NaN
